<a href="https://colab.research.google.com/github/Gregrs400/cmpsc472Project2/blob/set-fire/cmpsc472Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
%%writefile cmpsc472Project2.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/time.h>
#include <sys/resource.h>
#include <time.h>

/*
TODO:
- add global pool of resources to fight fires. This should be an int with an arbritrary value, e.g. 30 resources
- add fire fighting resources throughout the landscape
- determine whether to have multiple processes or multiple threads deal with the wildfires
- implement inter-process communication
- use semaphores/mutexes to lock critical sections
- have logic for extinguishing wildfires, add resources back to global resource pool when a fire is extinguished
*/


struct Fire {
  int row;
  int col;
  int priority;
  int requiredResources;
};


// Function to set a random position in the array to 1, which represents a wildfire.
struct Fire setFire(int array[20][20], int min, int max) {
  struct Fire fire;
  // Seed the random number generator
  srand(time(NULL));
  // Randomly select a row
  fire.row = rand() % (max - min + 1) + min;
  // Randomly select a column
  fire.col = rand() % (max - min + 1) + min;
  // Set the value at the selected row and column to 1
  array[fire.row][fire.col] = 1;
  printf("array value at position %d,%d: %d\n", fire.row, fire.col, array[fire.row][fire.col]);
  // Randomly assign a priority. Lower priorities have highest priority.
  fire.priority = rand() % (max - min + 1) + min;
  // Randomly assign a required number of resources to extinguish the fire.
  fire.requiredResources = rand() % (max - min + 1) + min;
  printf("Fire has priority %d and requires %d resources.\n", fire.priority, fire.requiredResources);
  return fire;
}

// Prints out the entire landscape array.
void printLandscape(int array[20][20]) {
  for (int i = 0; i < 20; i++) {
    for (int j = 0; j < 20; j++) {
      printf("%d, ", array[i][j]);
    }
    // Print a newline at the end of each row.
    printf("\n");
  }
}


int main() {
  // Define number of rows
  int rows = 20;
  // Define number of columns
  int columns = 20;
  // Create array to represent the landscape
  int landscape[rows][columns];
  // Fill array with 0's
  for (int i = 0; i < rows; i++) {
    for (int j = 0; j < columns; j++) {
      landscape[i][j] = 0;
    }
  }

  // Number of fires to be set.
  int numOfFires = 5;
  // Amount of time to wait between setting fires.
  int waitTime;
  // Create array of Fire structs to store info on each fire set.
  struct Fire fires[numOfFires];
  // For loop to set the fires.
  for (int j = 0; j < numOfFires; j++) {
    // Stores attributes (row, col, priority, number of required resources) about the fire in jth element of Fire struct array.
    fires[j] = setFire(landscape, 0, 19);
    printLandscape(landscape);
    printf("\n");

    // Seed random number generator
    srand(time(NULL));
    // Wait between 1 and 10 seconds before setting another fire.
    waitTime = rand() % (10 - 1 + 1) + 1;
    // Wait a random amount of time before setting another fire.
    sleep(waitTime);
  }

  return 0;
}


Overwriting cmpsc472Project2.c


In [125]:
%%shell
gcc cmpsc472Project2.c -o cmpsc472Project2
./cmpsc472Project2

array value at position 1,2: 1
Fire has priority 4 and requires 15 resources.
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
0, 0, 0